In [66]:
!pip install langgraph==0.4.7 jinja2 langchain-community langchain-openai fuzzysearch

In [67]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

from pydantic import BaseModel, Field
from operator import add
from typing import Annotated, Dict, Optional, Literal, Union

from langgraph.types import interrupt, Command

from langgraph.checkpoint.memory import MemorySaver

from enum import Enum

import json

from langgraph.graph import StateGraph, START, END

from jinja2 import Template

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY_2")

### Main Document

In [68]:
MAIN_DOCUMENT = r"""
# Слепая подпись Чаума и её использование в протоколе «Электронное голосование». Протокол «Игра в покер по переписке» (Ментальный покер)

---

## Введение

В этом материале рассмотрены два важных направления криптографии:

1. **Слепая подпись Чаума** — криптографический протокол, позволяющий получить подпись на сообщение, не раскрывая его содержания подписывающей стороне. Это обеспечивает анонимность и конфиденциальность, что особенно важно в электронном голосовании.

2. **Протокол «Игра в покер по переписке» (ментальный покер)** — распределённый криптографический протокол, позволяющий двум и более игрокам честно играть в покер удалённо, без доверенного посредника, сохраняя приватность карт и обеспечивая честность игры.

---

# Часть 1. Слепая подпись Чаума

---

## 1.1 Основные понятия и обозначения

- Рассматривается криптосистема RSA с ключами:
  - Модуль: \( N = p \cdot q \), произведение двух больших простых чисел.
  - Открытая экспонента: \( e \).
  - Секретная экспонента: \( d \), такая что \( ed \equiv 1 \pmod{\varphi(N)} \), где \(\varphi(N) = (p-1)(q-1)\).
- Сообщение для подписи: \( m \in \mathbb{Z}_N^* \) (обратимо по модулю \( N \)).
- Слепящий фактор (маска): \( r \in \mathbb{Z}_N^* \), случайное число, взаимно простое с \( N \).

---

## 1.2 Протокол слепой подписи Чаума

### Шаг 1. Слепление сообщения

Клиент выбирает случайный слепящий фактор \( r \) и вычисляет:

$$
m' = m \cdot r^e \pmod{N}
$$

Здесь \( m' \) — "слепленное" сообщение, которое отправляется подписывающей стороне. Благодаря умножению на \( r^e \), подписывающая сторона не видит исходное сообщение \( m \).

---

### Шаг 2. Подписание слепленного сообщения

Подписывающая сторона вычисляет подпись на слепленном сообщении:

$$
s' = (m')^d \pmod{N}
$$

и возвращает \( s' \) клиенту.

---

### Шаг 3. Удаление слепления

Клиент вычисляет подпись на исходном сообщении \( m \) как:

$$
s = s' \cdot r^{-1} \pmod{N}
$$

где \( r^{-1} \) — мультипликативный обратный к \( r \) по модулю \( N \).

---

### Шаг 4. Проверка подписи

Любой может проверить корректность подписи \( s \) на сообщении \( m \) с помощью открытого ключа:

$$
s^e \equiv m \pmod{N}
$$

---

## 1.3 Обоснование корректности протокола

Рассмотрим подробнее:

\[
\begin{aligned}
s' &= (m')^d = (m \cdot r^e)^d = m^d \cdot (r^e)^d = m^d \cdot r^{ed} \pmod{N} \\
s &= s' \cdot r^{-1} = m^d \cdot r^{ed} \cdot r^{-1} = m^d \cdot r^{ed - 1} \pmod{N}
\end{aligned}
\]

Поскольку \( ed \equiv 1 \pmod{\varphi(N)} \), по малой теореме Ферма:

\[
r^{ed} \equiv r \pmod{N}
\]

Следовательно,

\[
s = m^d \cdot r^{ed - 1} = m^d \cdot r^{0} = m^d \pmod{N}
\]

Таким образом, \( s \) — корректная подпись сообщения \( m \).

---

## 1.4 Свойства и безопасность

- **Анонимность:** Подписывающая сторона видит только слепленное сообщение \( m' \), не зная исходного \( m \).
- **Корректность:** После удаления слепления получается корректная подпись.
- **Защита от повторного использования:** Слепящий фактор \( r \) должен быть случайным и уникальным для каждого сообщения.
- **Безопасность:** Базируется на сложности факторизации \( N \) и безопасности RSA.

---

## 1.5 Использование слепой подписи Чаума в протоколе «Электронное голосование»

### Цель

Обеспечить анонимность голосующего при сохранении возможности проверить подлинность голоса и исключить двойное голосование.

---

### Идея протокола

1. Избиратель формирует голос \( m \) (например, код кандидата).
2. Выбирает случайный слепящий фактор \( r \) и вычисляет слепленное сообщение:

   $$
   m' = m \cdot r^e \pmod{N}
   $$

3. Отправляет \( m' \) в избирательную комиссию.
4. Комиссия подписывает \( m' \):

   $$
   s' = (m')^d \pmod{N}
   $$

5. Избиратель удаляет слепление:

   $$
   s = s' \cdot r^{-1} \pmod{N}
   $$

6. Избиратель отправляет пару \( (m, s) \) в систему голосования.
7. Система проверяет подпись:

   $$
   s^e \equiv m \pmod{N}
   $$

8. Голос засчитывается, при этом комиссия не знает, какой именно голос был подписан.

---

### Дополнительные меры

- Для предотвращения двойного голосования ведётся учёт уже выданных подписей.
- Для повышения анонимности могут использоваться mix-сети и другие криптографические методы.

---

# Часть 2. Протокол «Игра в покер по переписке» (Ментальный покер)

---

## 2.1 Постановка задачи

Обеспечить честную игру в покер между удалёнными игроками без доверенного посредника, при этом:

- Карты раздаются случайно и честно.
- Игроки не знают карты друг друга.
- Никакая сторона не может изменить или подделать карты.
- Все действия проверяемы.

---

## 2.2 Основные требования к протоколу

- **Случайность:** Карты должны быть случайно перемешаны.
- **Конфиденциальность:** Карты каждого игрока скрыты от других.
- **Честность:** Невозможность подделать карты.
- **Отсутствие доверенного посредника.**

---

## 2.3 Криптографические инструменты

- Используются асимметричные криптосистемы (например, RSA).
- Применяется **коммутативное шифрование** — шифр, для которого порядок шифрования не важен.
- Используются протоколы обмена ключами и коммитменты для обеспечения честности.

---

## 2.4 Коммутативное шифрование

### Определение

Шифр \( E_k \) называется коммутативным, если для любых ключей \( k_1, k_2 \) и сообщения \( m \):

$$
E_{k_1}(E_{k_2}(m)) = E_{k_2}(E_{k_1}(m))
$$

---

### Пример коммутативного шифра (на основе дискретного логарифма)

- Пусть \( G \) — группа порядка \( q \) с генератором \( g \).
- Ключи: \( k \in \mathbb{Z}_q \).
- Шифрование:

  $$
  E_k(m) = m^{k} \pmod{p}
  $$

- Расшифровка:

  $$
  D_k(c) = c^{k^{-1}} \pmod{p}
  $$

- Коммутативность:

  $$
  E_{k_1}(E_{k_2}(m)) = (m^{k_2})^{k_1} = m^{k_1 k_2} = (m^{k_1})^{k_2} = E_{k_2}(E_{k_1}(m))
  $$

---

## 2.5 Протокол ментального покера для двух игроков \( A \) и \( B \)

---

### Исходные данные

- Колода карт \( C = \{ c_1, c_2, \ldots, c_{52} \} \), представленных уникальными числами.
- Игрок \( A \) выбирает секретный ключ \( k_1 \).
- Игрок \( B \) выбирает секретный ключ \( k_2 \).
- Используется коммутативное шифрование \( E_k(\cdot) \).

---

### Шаги протокола

1. Игрок \( A \) шифрует всю колоду своим ключом и перемешивает:

   $$
   C_1 = \{ E_{k_1}(c_i) \}_{i=1}^{52}
   $$

2. Игрок \( A \) отправляет \( C_1 \) игроку \( B \).

3. Игрок \( B \) шифрует \( C_1 \) своим ключом и перемешивает:

   $$
   C_2 = \{ E_{k_2}(E_{k_1}(c_i)) \}_{i=1}^{52}
   $$

4. Игрок \( B \) отправляет \( C_2 \) обратно игроку \( A \).

5. Для выбора карты игрок \( A \) выбирает зашифрованную карту \( m \in C_2 \).

6. Игрок \( A \) удаляет своё шифрование:

   $$
   m' = D_{k_1}(m) = E_{k_2}(c_j)
   $$

7. Игрок \( A \) отправляет \( m' \) игроку \( B \).

8. Игрок \( B \) удаляет своё шифрование:

   $$
   c_j = D_{k_2}(m')
   $$

   Теперь карта \( c_j \) известна только игроку \( A \).

9. Аналогично игрок \( B \) выбирает свои карты.

---

## 2.6 Свойства протокола

- **Конфиденциальность:** Карты известны только их владельцам.
- **Честность:** Благодаря коммутативности шифрования никто не может изменить карты без обнаружения.
- **Отсутствие доверенного посредника:** Все операции выполняются между игроками напрямую.

---

## 2.7 Дополнительные меры безопасности

- Использование протоколов доказательства с нулевым разглашением для проверки честности перемешивания.
- Протоколы коммитмента для предотвращения подделки карт.
- Механизмы для предотвращения повторного использования карт.

---

# Итог

---

- **Слепая подпись Чаума** — протокол, позволяющий получить подпись на сообщение, не раскрывая его содержания, что обеспечивает анонимность и используется в электронном голосовании.
- **Протокол ментального покера** — распределённый криптографический протокол, позволяющий честно играть в покер по переписке без доверенного посредника, используя коммутативное шифрование.
- Ключевые математические операции основаны на свойствах RSA и коммутативных шифров, обеспечивающих безопасность, анонимность и честность.
- Дополнительные криптографические методы (хеш-функции, коммитменты, доказательства с нулевым разглашением) повышают надёжность и защищают от мошенничества.

---

# Резюме для экзамена

- Опишите протокол слепой подписи Чаума с формулами:

  $$
  m' = m \cdot r^e \pmod{N}, \quad s' = (m')^d \pmod{N}, \quad s = s' \cdot r^{-1} \pmod{N}, \quad s^e \equiv m \pmod{N}
  $$

- Объясните, как слепая подпись обеспечивает анонимность в электронном голосовании.
- Раскройте идею ментального покера, основанного на коммутативном шифровании.
- Приведите пример протокола для двух игроков с подробным описанием шагов.
- Объясните важность коммутативности шифра для честности протокола.
- Укажите современные методы повышения безопасности (доказательства с нулевым разглашением, коммитменты).
"""

### System Prompt

In [84]:
SYSTEM_PROMPT_TEMPLATE = Template(r"""
KEYWORD: cryptography, mathematics, explanation

<role>
You are a university-level cryptography professor, whose mission is to produce a **thorough, comprehensive, and absolutely exhaustive educational material** on the given exam question. The student must gain a 100% understanding of the topic by studying your material. If the student presents your content as their answer, the examiner should have no follow-up questions and should immediately award the highest grade ("excellent").
</role>

<current_phase>
**Current Phase**: You are now in an iterative refinement cycle, improving the previously generated educational material (between <generated_material></generated_material> XML-tags) based on feedback from your student (user) who is preparing for an exam.
</current_phase>

<generated_material>
{{ generated_material }}
</generated_material>

<editing_standards>
When making edits to the educational material, ensure that your modifications maintain and enhance the following qualities:

1. **Comprehensiveness**: Every edit must preserve or improve the material's coverage of all necessary angles — definitions, context, theory, practical relevance and connections to real cryptographic algorithms. If the student's feedback reveals a gap, fill it completely.
2. **Clarity**: Replace any unclear or ambiguous passages with precise, unambiguous language. Each edit should make the logic more crystal-clear than before. Never introduce vagueness when fixing other issues.
3. **Mathematics**: When editing or writing new mathematical content:
   - If a formula lacks proper derivation, add **detailed, step-by-step derivation**
   - If symbols are unexplained, add their definitions
   - If practical application is missing, add explanation of **how this formula or principle is used in practical cryptographic algorithms**
   - Strengthen the connection between mathematical theory and real-world cryptography in your edits
   - Maintain proper formatting:
     - Inline formulas: `$c = 3 \times 10^8\ m/s$`
     - Display formulas:
     $$
     \vec{F} = m \vec{a} = m \frac{d\vec{v}}{dt}
     $$
   - Regardless of the source text, every mathematical expression in the output **must** be enclosed in LaTeX math delimiters.
4. **Examples and Intuition**: When the material lacks concrete understanding:
   - Add intuitive analogies where abstract concepts need clarification
   - Insert illustrative examples where theory needs grounding
   - Create text-based diagrams where visual representation would help
5. **Gap Elimination**: Each edit should identify and eliminate "white spots" in the material. If your edit addresses one issue but creates another gap, expand the edit to be comprehensive.
6. **Self-sufficiency**: Ensure every edit maintains the material's self-contained nature. Don't reference external concepts without explaining them within the material itself.
7. **Language consistency**: All edits must be in Russian, maintaining the academic style and terminology appropriate for the subject.
8. **Target audience calibration**: Remember that edits are for advanced students (graduate/master's/PhD level):
   - Remove or condense any basic explanations that slipped into the material
   - Enhance focus on advanced aspects, recent research, and deep mathematical structure
   - Strengthen discussion of practical subtleties that only advanced practitioners would appreciate
   - Assume familiarity with core cryptographic terminology — don't over-explain basics
</editing_standards>

<editing_instructions>
1. **Understand User Intent**: Carefully analyze the student's feedback to understand exactly what needs to be improved or changed in the material.
2. **Precise Text Matching**: The `old_text` field must match **exactly** a fragment of text from the <generated_material></generated_material> section, including all punctuation, spacing, and formatting. Even a single character difference will cause an error.
3. **Maintain Quality**: When making edits, maintain the same high academic standard and comprehensive depth as the original material. Your edits should enhance, not diminish, the educational value.
4. **Autonomous Editing**: When you identify multiple improvements needed:
   - Use `continue_editing: true` to make consecutive edits without waiting for user input
   - Set `continue_editing: false` when you need user clarification or confirmation before proceeding
5. **Communication with Student**: Use the "message" action when:
   - You need clarification about the student's requirements
   - You need additional information to proceed
6. **Completion Signal**: Use the "complete" action when:
   - The student explicitly indicates they are satisfied with the material
   - The student signals (explicitly or implicitly) that they have no more edits
   - All requested improvements have been implemented
7. **Error Handling**: If you receive an error message in the conversation history indicating incorrect arguments, carefully review the exact text fragment you're trying to replace and ensure perfect character-by-character matching.
</editing_instructions>
""")

### Graph

In [72]:
def pretty_print_pydantic(pydantic_model):
    """
    Возвращает красиво отформатированную JSON-схему Pydantic-модели.

    Args:
        pydantic_model: Pydantic-модель

    Returns:
        str: JSON-схема модели с отступами
    """
    return json.dumps(pydantic_model.model_json_schema(), indent=4, ensure_ascii=False)

In [73]:
class GeneralState(BaseModel):
    document: str = Field(
        description="Current version of the document being edited"
    )
    messages: list = Field(
        default_factory=list,
        description="Conversation history between user and agent"
    )
    needs_user_input: bool = Field(
        default=True,
        description="Whether to request user input on current iteration"
    )
    last_action: Optional[str] = Field(
        default=None,
        description="Type of last action performed (edit/message/complete)"
    )
    edit_count: int = Field(
        default=0,
        description="Total number of edits performed"
    )
    agent_message: Optional[str] = Field(
        default=None,
        description="Message from agent to display to user"
    )

class ActionDecision(BaseModel):
    """Step 1: Decide what action to take"""
    action_type: Literal["edit", "message", "complete"] = Field(description="Type of action to perform")
    # reasoning: Optional[str] = Field(
    #     default=None,
    #     description="Brief reasoning for this decision"
    # )

# Схемы для каждого типа действия (Step 2)
class EditDetails(BaseModel):
    """Details for edit action"""
    old_text: str = Field(description="Exact text to replace")
    new_text: str = Field(description="Replacement text")
    continue_editing: bool = Field(
        default=True,
        description="Continue editing autonomously after this edit"
    )

class MessageDetails(BaseModel):
    """Details for message action"""
    content: str = Field(description="Message to send to user")

In [75]:
model = ChatOpenAI(model_name="gpt-4.1", temperature=0)

In [77]:
from fuzzysearch import find_near_matches
from typing import Tuple, Optional

def fuzzy_find_and_replace(
    document: str,
    target: str,
    replacement: str,
    threshold: float = 0.85
) -> Tuple[str, bool, Optional[str], float]:
    """
    Нечеткий поиск и замена с базовой защитой от edge cases.

    Returns: (new_document, success, found_text, similarity)
    """

    # Edge case: пустые строки
    if not target or not document:
        return document, False, None, 0.0

    # Для коротких строк - только точное совпадение
    if len(target) < 10:
        if target in document:
            idx = document.index(target)
            new_doc = document[:idx] + replacement + document[idx + len(target):]
            return new_doc, True, target, 1.0
        return document, False, None, 0.0

    # Вычисляем дистанцию (минимум 1 для избежания точного поиска)
    max_distance = max(1, int(len(target) * (1 - threshold)))

    # Для очень длинных строк ограничиваем дистанцию
    if len(target) > 100:
        max_distance = min(max_distance, 15)

    # Поиск
    try:
        matches = find_near_matches(
            target,
            document,
            max_l_dist=max_distance
        )
    except Exception:
        # На случай если fuzzysearch выдаст ошибку
        return document, False, None, 0.0

    if not matches:
        return document, False, None, 0.0

    # Берем первое совпадение
    match = matches[0]

    # Безопасное вычисление similarity
    if len(target) > 0:
        similarity = max(0.0, 1 - (match.dist / len(target)))
    else:
        similarity = 1.0 if match.dist == 0 else 0.0

    # Заменяем
    new_document = (
        document[:match.start] +
        replacement +
        document[match.end:]
    )

    return new_document, True, match.matched, similarity

('## 1.1 Новый заголовок', True, '## 1.1 Основные понятия и обозначения\n\n- Рассматривается криптосистема RSA с ключами:\n  - Модуль: \\( N = p \\cdot q \\), произведение двух больших простых чисел.', 0.9936708860759493)


In [ ]:
# Пример использования
document = """## 1.1 Основные понятия и обозначения

- Рассматривается криптосистема RSA с ключами:
  - Модуль: \( N = p \cdot q \), произведение двух больших простых чисел."""

old_text = """## 1.1 Основные понятия и обозначения

- Рассматривается криптосистема RSA с ключами:
 - Модуль: \\( N = p \\cdot q \\), произведение двух больших простых чисел."""

new_text = "## 1.1 Новый заголовок"

# Используем fuzzysearch
answer = fuzzy_find_and_replace_fast(
    document, old_text, new_text, max_distance=5
)

print(answer)

new_doc, success, found, similarity = answer

In [78]:
from difflib import SequenceMatcher

def inline_diff_markers(target: str, found: str) -> str:
    """Показывает различия прямо в тексте с маркерами"""

    s = SequenceMatcher(None, target, found)

    target_marked = []
    found_marked = []

    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'equal':
            target_marked.append(target[i1:i2])
            found_marked.append(found[j1:j2])
        elif tag == 'replace':
            target_marked.append(f"[{target[i1:i2]}]")
            found_marked.append(f"[{found[j1:j2]}]")
        elif tag == 'delete':
            target_marked.append(f"[{target[i1:i2]}]")
            found_marked.append("[MISSING]")
        elif tag == 'insert':
            target_marked.append("[MISSING]")
            found_marked.append(f"[{found[j1:j2]}]")

    return f"""
SEARCH: {''.join(target_marked)}
FOUND:  {''.join(found_marked)}

Brackets [] indicate mismatched parts."""

# Пример
target = "The cat is sleeping on the red sofa"
found = "The dog is sleeping on a blue sofa"

print(inline_diff_markers(target, found))
# SEARCH: The [cat] is sleeping on [the] [red] sofa
# FOUND:  The [dog] is sleeping on [a] [blue] sofa


SEARCH: The [cat] is sleeping on [th]e[ red] sofa
FOUND:  The [dog] is sleeping on [a blu]e[MISSING] sofa

Brackets [] indicate mismatched parts.


In [79]:
def xml_style_diff(target: str, found: str) -> str:
    """Использует XML-подобные теги для указания различий"""

    s = SequenceMatcher(None, target, found)

    result = []
    result.append("COMPARISON RESULT:\n")

    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag == 'equal':
            result.append(target[i1:i2])
        elif tag == 'replace':
            result.append(f"<diff expected='{target[i1:i2]}' found='{found[j1:j2]}' />")
        elif tag == 'delete':
            result.append(f"<missing>{target[i1:i2]}</missing>")
        elif tag == 'insert':
            result.append(f"<extra>{found[j1:j2]}</extra>")

    return ''.join(result)

# Пример
print(xml_style_diff("The cat is sleeping", "The dog is sleeping"))
# COMPARISON RESULT:
# The <diff expected='cat' found='dog' /> is sleeping

COMPARISON RESULT:
The <diff expected='cat' found='dog' /> is sleeping


In [80]:
def position_based_diff(target: str, found: str) -> str:
    """Указывает точные позиции различий"""

    s = SequenceMatcher(None, target, found)

    feedback = []
    feedback.append(f"TARGET: {target}")
    feedback.append(f"FOUND:  {found}")
    feedback.append("\nMISMATCHES:")

    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if tag != 'equal':
            feedback.append(
                f"  Position {i1}-{i2}: Expected '{target[i1:i2] if i1 < i2 else 'nothing'}', "
                f"but found '{found[j1:j2] if j1 < j2 else 'nothing'}'"
            )

    return '\n'.join(feedback)

# Пример
print(position_based_diff("The cat is sleeping", "The dog is sleeping"))
# TARGET: The cat is sleeping
# FOUND:  The dog is sleeping
#
# MISMATCHES:
#   Position 4-7: Expected 'cat', but found 'dog'

TARGET: The cat is sleeping
FOUND:  The dog is sleeping

MISMATCHES:
  Position 4-7: Expected 'cat', but found 'dog'


In [82]:
def main_node(state: GeneralState) -> Command[Literal["main_node", "__end__"]]:
    messages = state.messages

    # print(f"--- Entering main_node. Current state: {state}") # Debug print

    # Запрашиваем ввод пользователя только если нужно
    if state.needs_user_input:

        msg_to_user = state.agent_message or "Какие правки внести? "

        # Получаем ввод пользователя
        print(f"--- Interrupting for user input. Message: {msg_to_user}") # Debug print
        user_feedback = interrupt(value=msg_to_user)
        print(f"--- User feedback received: {user_feedback}") # Debug print

        if user_feedback:
            messages.append(HumanMessage(content=user_feedback))

            return Command(
                goto="main_node",
                update={
                    "messages": messages,
                    "agent_message": None,  # Обнуляем сообщение агента
                    "needs_user_input": False  # Переходим к обработке
                }
            )
    # Шаг 1: Определяем тип действия
    print("--- Invoking model for ActionDecision") # Debug print
    decision = model.with_structured_output(ActionDecision).invoke(
        [SystemMessage(content=SYSTEM_PROMPT_TEMPLATE.render(generated_material=state.document))] + messages)

    print(f"decision: {decision.model_dump_json()}")

    messages.append(AIMessage(content=decision.model_dump_json()))

    # Шаг 2: Получаем детали в зависимости от типа
    if decision.action_type == "edit":
        print("--- Action type is 'edit'. Invoking model for EditDetails") # Debug print
        details = model.with_structured_output(EditDetails).invoke(
            [SystemMessage(content=SYSTEM_PROMPT_TEMPLATE.render(generated_material=state.document))] + messages)
        print("details: ", details)
        return handle_edit_action(state, details, messages)

    elif decision.action_type == "message":
        print("--- Action type is 'message'. Invoking model for MessageDetails") # Debug print
        details = model.with_structured_output(MessageDetails).invoke(
            [SystemMessage(content=SYSTEM_PROMPT_TEMPLATE.render(generated_material=state.document))] + messages)
        print("details: ", details)
        return handle_message_action(state, details, messages)

    elif decision.action_type == "complete":
        print("--- Action type is 'complete'. Handling completion.") # Debug print
        return handle_complete_action(state)


def handle_edit_action(state: GeneralState, action: EditDetails, messages: list) -> Command:
    """Обработка действия редактирования с использованием нечеткого поиска."""
    document = state.document
    old_text = action.old_text
    new_text = action.new_text

    print(f"--- Attempting to edit. old_text (first 50 chars): '{old_text[:50]}...', new_text (first 50 chars): '{new_text[:50]}...'") # Debug print

    # Используем fuzzy_find_and_replace
    new_document, success, found_text, similarity = fuzzy_find_and_replace(
        document, old_text, new_text
    )

    if not success:
        # Текст не найден с достаточным сходством
        error_msg = f"Error: Text similar to '{old_text[:50]}...' not found in document (similarity: {similarity:.2f})."
        print(f"--- Edit failed: Text not found. Error message: {error_msg}") # Debug print
        messages.append(SystemMessage(
            content=f"[EDIT ERROR]: {error_msg}"
        ))
        if found_text:
             messages.append(SystemMessage(
                content=f"[DEBUG]: Closest match found (similarity: {similarity:.2f}):\n{inline_diff_markers(old_text, found_text)}"
             ))


        return Command(
            goto="main_node",
            update={
                "messages": messages,
                "needs_user_input": False,  # Запрашиваем ввод после ошибки
                "last_action": "edit_error"
            }
        )

    else:
        # Успешное редактирование
        print("--- Fuzzy edit successful. Performing replacement.") # Debug print
        document = new_document # Обновляем документ с результатом нечеткой замены
        edit_count = state.edit_count + 1

        # Логируем успешную правку, указывая найденный текст и сходство
        success_msg = f"Edit #{edit_count} applied successfully (similarity: {similarity:.2f})"
        print(f"--- {success_msg}") # Debug print
        messages.append(SystemMessage(
            content=f"[EDIT SUCCESS #{edit_count}]: Replaced text (similarity: {similarity:.2f})."
        ))
        # Показываем, что было найдено и чем заменено
        messages.append(SystemMessage(
            content=f"[DEBUG]: Replaced:\n'{found_text[:50]}...'\nWith:\n'{new_text[:50]}...'"
        ))


        # Определяем нужен ли пользовательский ввод
        needs_input = not action.continue_editing
        print(f"--- Continue editing: {action.continue_editing}, Needs user input: {needs_input}") # Debug print

        # Если не продолжаем автономно, показываем что сделали
        agent_msg = None
        if not action.continue_editing:
            agent_msg = f"Я внёс правки, что дальше?"
            print(f"--- Setting agent message: {agent_msg}") # Debug print


        return Command(
            goto="main_node",
            update={
                "document": document,
                "messages": messages,
                "needs_user_input": needs_input,
                "edit_count": edit_count,
                "agent_message": agent_msg,
                "last_action": "edit"
            }
        )


def handle_message_action(state: GeneralState, action: MessageDetails, messages: list) -> Command:
    """Обработка сообщения пользователю"""
    print(f"--- handle_message_action called with content: '{action.content[:50]}...'") # Debug print
    # Добавляем сообщение агента в историю
    messages.append(AIMessage(content=action.content))

    return Command(
        goto="main_node",
        update={
            "messages": messages,
            "needs_user_input": True,  # Всегда ждем ответ после сообщения
            "agent_message": action.content,
            "last_action": "message"
        }
    )


def handle_complete_action(state: GeneralState) -> Command:
    """Завершение редактирования"""
    print("--- handle_complete_action called. Saving document.") # Debug print
    # Сохраняем финальный документ
    with open("updated_document.md", "w", encoding="utf-8") as f:
        f.write(state.document)

    return Command(
        goto=END,
        update={
            "messages": [],  # Очищаем историю
            "last_action": "complete"
        }
    )

In [85]:
# --- Создание графа LangGraph ---
workflow = StateGraph(GeneralState)

workflow.add_node("main_node", main_node)

workflow.set_entry_point("main_node")

checkpointer = MemorySaver()

graph = workflow.compile(checkpointer=checkpointer)

#==========================================================

config = {"configurable": {"thread_id": "1"}}

while True:
    state = await graph.aget_state(config)

    if state.values:
        input_state = Command(resume=user_feedback)
    else:
        input_state = {
            "document": MAIN_DOCUMENT
            }

    async for event in graph.astream(
        input_state, config, stream_mode="updates"
    ):
        print(event)
        print("=" * 50)
        # Check if the document has been updated in this event
        if 'main_node' in event:
            if "document" in event['main_node']:
                updated_document = event['main_node']["document"]
                with open("updated_document.md", "w", encoding="utf-8") as f:
                    f.write(updated_document)
                print("Document updated and saved to updated_document.md")

    if "__interrupt__" in event:
        user_feedback = input(event['__interrupt__'][0].value)
    else:
        break

--- Interrupting for user input. Message: Какие правки внести? 
{'__interrupt__': (Interrupt(value='Какие правки внести? ', resumable=True, ns=['main_node:ceebd2e4-a158-54c6-c31a-a9cf406cc82f']),)}
Какие правки внести? Перепиши секцию 1.1
--- Interrupting for user input. Message: Какие правки внести? 
--- User feedback received: Перепиши секцию 1.1
{'main_node': {'messages': [HumanMessage(content='Перепиши секцию 1.1', additional_kwargs={}, response_metadata={})], 'agent_message': None, 'needs_user_input': False}}
--- Invoking model for ActionDecision
decision: {"action_type":"edit"}
--- Action type is 'edit'. Invoking model for EditDetails
details:  old_text='## 1.1 Основные понятия и обозначения\n\n- Рассматривается криптосистема RSA с ключами:\n  - Модуль: \\( N = p \\cdot q \\), произведение двух больших простых чисел.\n  - Открытая экспонента: \\( e \\).\n  - Секретная экспонента: \\( d \\), такая что \\( ed \\equiv 1 \\pmod{\\varphi(N)} \\), где \\(\\varphi(N) = (p-1)(q-1)\\).\n-

CancelledError: 